# JData

## 数据整理部分

先引入需要的库

In [2]:
import numpy as np
import pandas as pd

#### 整理用户数据。

把年龄规范化。发现数据里有15岁以下、16-25岁、26-35岁、36-45岁、46-55岁、-1这几种数据。将它们定义为：

{-1:-1,'15岁以下':0,'16-25岁':1,'26-35岁':2,'36-45岁':3,'46-55岁':4,'56岁以上':5}

同时将3个数据异常的账户进行处理(数据为NULL的重新定义为-1)

In [3]:
def deal_age(str):
    return str.replace("15岁以下","0").replace("16-25岁","1").replace("26-35岁","2").replace("36-45岁","3").replace("46-55岁","4").replace("56岁以上","5")

g=file('./data/JData_User_new.csv','w+')
with open('./data/JData_User.csv','r') as f:
    line = f.readline()
    while line:
        line = deal_age(line).replace("NULL","-1")
        g.writelines(line)
        line = f.readline()
    g.close()
!rm ./data/JData_User.csv
!mv ./data/JData_User_new.csv ./data/JData_User.csv

#### 将数据载入pandas
(如果前面不处理会报错)

In [4]:
action_files = ["./data/JData_Action_201602.csv","./data/JData_Action_201603.csv","./data/JData_Action_201604.csv"]
comment_file = "./data/JData_Comment.csv"
product_file = "./data/JData_Product.csv"
user_file = "./data/JData_User.csv"

加载大文件按照 http://blog.csdn.net/shuimuqingyi/article/details/44569435 的方法分chunk处理。


加载大概需要30秒

In [102]:
#加载行为数据
chunk_size = 1000000
action_data = []

for f in action_files:
    reader = pd.read_csv(f, iterator=True)
    chunk_loop = True
    while chunk_loop:
        try:
            action_data.append(reader.get_chunk(chunk_size))
        except StopIteration:
            chunk_loop = False
            print"Iteration is stopped, loaded " + f

action_data = pd.concat(action_data, ignore_index=True)
print action_data.head(5)

Iteration is stopped, loaded ./data/JData_Action_201602.csv
Iteration is stopped, loaded ./data/JData_Action_201603.csv
Iteration is stopped, loaded ./data/JData_Action_201604.csv
    user_id  sku_id                 time  model_id  type  cate  brand
0  266079.0  138778  2016-01-31 23:59:02       NaN     1     8    403
1  266079.0  138778  2016-01-31 23:59:03       0.0     6     8    403
2  200719.0   61226  2016-01-31 23:59:07       NaN     1     8     30
3  200719.0   61226  2016-01-31 23:59:08       0.0     6     8     30
4  263587.0   72348  2016-01-31 23:59:08       NaN     1     5    159


In [103]:
print action_data.dtypes

user_id     float64
sku_id        int64
time         object
model_id    float64
type          int64
cate          int64
brand         int64
dtype: object


In [104]:
#处理数据结构，并将为NaN的model_id更换为-1
action_data["model_id"] = action_data["model_id"].fillna(value = -1.0)

action_data[["user_id","model_id"]] = action_data[["user_id","model_id"]].astype(int)
print action_data.head(5)
print "----"
print action_data.dtypes

   user_id  sku_id                 time  model_id  type  cate  brand
0   266079  138778  2016-01-31 23:59:02        -1     1     8    403
1   266079  138778  2016-01-31 23:59:03         0     6     8    403
2   200719   61226  2016-01-31 23:59:07        -1     1     8     30
3   200719   61226  2016-01-31 23:59:08         0     6     8     30
4   263587   72348  2016-01-31 23:59:08        -1     1     5    159
----
user_id      int64
sku_id       int64
time        object
model_id     int64
type         int64
cate         int64
brand        int64
dtype: object


In [108]:
#导入用户信息
user_data = pd.read_csv(user_file)
print user_data.head(5)
print "现在有" + str(user_data["user_id"].count()) + "条用户数据"

   user_id  age  sex  user_lv_cd user_reg_tm
0   200001    5    2           5  2016-01-26
1   200002   -1    0           1  2016-01-26
2   200003    3    1           4  2016-01-26
3   200004   -1    2           1  2016-01-26
4   200005    1    0           4  2016-01-26
现在有105321条用户数据


分析用户数据，大概要用10分钟

In [114]:
#根据type统计数量
from collections import Counter
ad_backup = action_data.copy(deep = True)#拷贝一份行为输出出来，用于做用户数据分析。

ad_backup = ad_backup.groupby(["user_id"],as_index=False)

def countUserAction(group):
    #将行为数据按用户分类统计
    behave_counter = Counter(group["type"])
    group["browse"] = behave_counter[1]#浏览数
    group["cart"] = behave_counter[2]#加入购物车数
    group["delcart"] = behave_counter[3]#从购物车删除数
    group["buy"] = behave_counter[4]#购买数
    group["favorite"] = behave_counter[5]#收藏数
    group["click"] = behave_counter[6]#点击数
    return group[["user_id","browse","cart","delcart","buy","favorite","click"]]
    
ad_backup = ad_backup.apply(countUserAction)

In [115]:
#太慢了，先备份一下
ad_backup2 = ad_backup.copy(deep = True)

In [116]:
#去掉相同的user_id
ad_backup2 = ad_backup2.drop_duplicates("user_id")
print "现在有" + str(ad_backup2["user_id"].count()) + "条数据"

现在有105180条数据


删除没有进行过操作的用户（添加购物车数、购买数、收藏数为0）

In [117]:
ad_backup2 = ad_backup2[(ad_backup2["cart"]!=0)|(ad_backup2["buy"]!=0)|(ad_backup2["favorite"]!=0)]
print "现在有" + str(ad_backup2["user_id"].count()) + "条数据"

现在有76982条数据


将用户信息与以上数据合并，存入一个csv文件中。

In [129]:
user_data = pd.merge(user_data,ad_backup2,on="user_id")
user_data.to_csv("./data/activity_user_data.csv", index=False)

同样，将行为数据根据以上数据进行筛选

In [132]:
action_data_col = action_data.columns
print "现在共有"+str(action_data["user_id"].count())+"条行为数据"
cleaned_action_data = pd.merge(action_data,ad_backup2,on="user_id")[action_data_col]
print "清理后还剩"+str(cleaned_action_data["user_id"].count())+"条数据"

现在共有37347993条行为数据
清理后还剩34157595条数据


将新的行为数据用csv文件保存。

In [134]:
cleaned_action_data.to_csv("./data/action_data.csv", index=False)

然后官方数据就没用了，早点删了腾硬盘。。

In [135]:
!rm ./original_data/JData*
!rm ./data/JData_Action*
!rm ./data/JData_User.csv

然后处理商品和评论信息

In [140]:
product_data = pd.read_csv(product_file)[["sku_id","cate","brand"]]
comment_data = pd.read_csv(comment_file)
print "有"+str(product_data["sku_id"].count())+"条商品记录"
print "有"+str(comment_data["sku_id"].count())+"条评论记录"

有24187条商品记录
有558552条评论记录


In [156]:
product_data = pd.merge(product_data,comment_data,on="sku_id",how="left")
product_data.to_csv("./data/product_data.csv",index=False)

In [157]:
!rm ./data/JData*

现在所有的数据都已经被整理为三个csv文件，分别为

商品与评价数据 product_data.csv(3.3MB)

活跃用户数据 activity_user_data.csv(3MB)

行为数据 action_data.csv(1.51GB)


----

它们的数据格式分别为：

商品与评价数据

| sku_id           | cate | brand | dt | comment_num | has_bad_comment | bad_comment_rate |
|:-------------:|:------:|:------:|:------:|:------:|:------:|:------:|
| 商品id | 分类      |品牌      |统计截止日期      |评价数量      | 是否有差评      | 差评率      |

其中评价数量含义为
0表示无评论，1表示有1条评论，

 2表示有2-10条评论，
 
 3表示有11-50条评论，
 
 4表示大于50条评论


----
活跃用户数据

| user_id    | age | sex | user_lv_cd | user_reg_tm | browse | cart |delcart |buy |favorite |click |
|:-------------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
| 用户id | 年龄      |性别      |用户等级      |注册日期      | 查看商品次数      | 添加购物车次数      |删除购物车次数      |下单次数      |收藏次数      |点击次数      |


其中年龄

-1表示未知，

0代表15岁以下，

1代表16-25岁

2代表26-35岁

3代表36-45岁

4代表46-55岁



性别：
0表示男，1表示女，2表示保密


----
行为数据

user_id,sku_id,time,model_id,type,cate,brand

| user_id           | sku_id | time | model_id | type | cate | brand |
|:-------------:|:------:|:------:|:------:|:------:|:------:|:------:|
| 用户id | 商品id      |时间      |点击模块      |行为类型      | 分类      | 品牌      |

其中行为类型为
1.浏览（指浏览商品详情页）；
 2.加入购物车；3.购物车删除；4.下单；5.关注；6.点击

In [159]:
print Counter(user_data["sex"])

Counter({2: 38024, 0: 32761, 1: 6194, -1: 3})


发现用户数据中性别有-1的，手动改成了2.